In [1]:
print("Hello Pydantic")


Hello Pydantic


[How to Make the Most of Pydantic](https://towardsdatascience.com/how-to-make-the-most-of-pydantic-aa374d5c12d)

In [2]:
from pydantic import BaseModel, ValidationError, Extra, Field
from functools import singledispatch
from typing import Optional
from enum import Enum
import time
from datetime import datetime


In [3]:
class Address(BaseModel):
    """
    Cat API Address definition
    """
    city: str
    zip_code: str
    number: int


class CatRequest(BaseModel):
    """
    Cat API Request definition
    """
    name: str
    age: int
    address: Address
 

In [4]:
my_json = {
    "name": "Lévy",
    "age": 3,
    "address": {
        "city": "Wonderland",
        "zip_code": "ABCDE",
        "number": 123
    }
}


In [5]:
my_json


{'name': 'Lévy',
 'age': 3,
 'address': {'city': 'Wonderland', 'zip_code': 'ABCDE', 'number': 123}}

In [6]:
data = CatRequest.parse_obj(my_json)


In [7]:
data


CatRequest(name='Lévy', age=3, address=Address(city='Wonderland', zip_code='ABCDE', number=123))

In [8]:
data.name  # Lévy


'Lévy'

In [9]:
data.address.number  # 123


123

In [10]:

bad_data = {
    "name": "Lévy",
    "age": "am I an age?",  # Note the type change here
    "address": {
        "city": "Wonderland",
        "zip_code": "ABCDE",
        "number": 123
    }
}


In [11]:
bad_data


{'name': 'Lévy',
 'age': 'am I an age?',
 'address': {'city': 'Wonderland', 'zip_code': 'ABCDE', 'number': 123}}

In [12]:
try:
    CatRequest.parse_obj(bad_data)
except ValidationError as err:
    print("Something went wrong with the data!")


Something went wrong with the data!


In [13]:
unnecessary_data = {
    "name": "Lévy",
    "age": 3,
    "key": "value",  # unnecessary
    "key2": "value2",  # unnecessary x2
    "address": {
        "city": "Wonderland",
        "zip_code": "ABCDE",
        "number": 123
    }
}


In [14]:
unnecessary_data


{'name': 'Lévy',
 'age': 3,
 'key': 'value',
 'key2': 'value2',
 'address': {'city': 'Wonderland', 'zip_code': 'ABCDE', 'number': 123}}

In [15]:

data = CatRequest.parse_obj(unnecessary_data)


In [16]:
data


CatRequest(name='Lévy', age=3, address=Address(city='Wonderland', zip_code='ABCDE', number=123))

In [17]:
class Address(BaseModel):
    """
    Cat API Address definition
    """
    class Config:
        extra = Extra.forbid

    # Note how we can even add descriptions to the fields!
    city: str = Field(..., description="Where the cat lives")
    zip_code: str
    number: int


class CatRequest(BaseModel):
    """
    Cat API Request definition
    """
    class Config:
        extra = Extra.forbid

    name: str
    age: int
    address: Address


In [18]:
try:
    data = CatRequest.parse_obj(unnecessary_data)
except Exception as e:
    print(e)


2 validation errors for CatRequest
key
  extra fields not permitted (type=value_error.extra)
key2
  extra fields not permitted (type=value_error.extra)


In [19]:

@singledispatch
def process(model):
    """
    Default processing definition
    """
    raise NotImplementedError(f"I don't know how to process {type(model)}")


@process.register
def _(model: Address):
    """
    Handle addresses
    """
    print(f"Just got an address from {model.city}")


@process.register
def _(model: CatRequest):
    """
    Handle Cat Requests
    """
    print(f"Processing {model.name} the cat!")


In [20]:
address = Address(
    city="Wonderland",
    zip_code="ABCDE",
    number=123,
)

In [21]:
cat = CatRequest(
    name="Lévy",
    age=3,
    address=address
)

In [22]:

process(address)  # Just got an address from Wonderland


Just got an address from Wonderland


In [23]:
process(cat)  # Processing Lévy the cat!

Processing Lévy the cat!


In [24]:
# NotImplementedError: I don't know how to process <class 'str'>

try:
    process("something else")
except Exception as e:
    print(e)

I don't know how to process <class 'str'>


[Introduction to Pydantic for FastAPI](https://www.fastapitutorial.com/blog/introduction-pydantic-for-fastapi/)

In [25]:
class Blog(BaseModel):
    title: str 
    description: Optional[str]=None
    is_active: bool

In [26]:
Blog(title="My First Blog", is_active=True)
# Output : Blog(title='My First Blog', description=None, is_active=True)


Blog(title='My First Blog', description=None, is_active=True)

In [27]:
class Languages(str, Enum):
    PY = "Python"
    JAVA = "Java"
    GO = "Go"


class Blog(BaseModel):
    title: str 
    language : Languages = Languages.PY
    is_active: bool

In [28]:

Blog(title="My First Blog", language="Java", is_active=True)
# Output: Blog(title='My First Blog', language=<Languages.JAVA: 'Java'>, is_active=True)

Blog(title='My First Blog', language=<Languages.JAVA: 'Java'>, is_active=True)

In [29]:

try:
    Blog(title="My First Blog", language="C++", is_active=True)
except Exception as e:
    print(e)
# Output: ValidationError: 1 validation error for Blog language value is not a valid enumeration member; permitted: 'Python', 'Java', 'Go' 

1 validation error for Blog
language
  value is not a valid enumeration member; permitted: 'Python', 'Java', 'Go' (type=type_error.enum; enum_values=[<Languages.PY: 'Python'>, <Languages.JAVA: 'Java'>, <Languages.GO: 'Go'>])


In [30]:
class Blog(BaseModel):
    title: str 
    created_at: datetime = Field(default_factory=datetime.now)
    is_active: bool

In [31]:
print(Blog(title="Our First Blog", is_active=True))

title='Our First Blog' created_at=datetime.datetime(2022, 12, 28, 20, 35, 35, 830937) is_active=True


In [32]:
time.sleep(4)
print(Blog(title="Our Second Blog", is_active=True))

title='Our Second Blog' created_at=datetime.datetime(2022, 12, 28, 20, 35, 39, 854226) is_active=True
